In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [ ]:

import sys
sys.path.append('../utils')
import functions

In [ ]:
df = pd.read_csv('../data/local/raw/doesthedogdie.com.csv')
df.head()

In [ ]:
functions.show_basic_info(df)

Round to 3 decimals in 'yes_ratio'

In [ ]:
df['yes_ratio'] = df['yes_ratio'].round(3)

Check if there are duplicate titles

In [ ]:
duplicates = df[df['title'].duplicated()]

duplicate_count = df['title'].duplicated().sum()

print(f"Number of duplicates in 'title': {duplicate_count}")

### Check the categories in 'title' column

In [ ]:
# Extract the unique values before the underscore in 'title'
unique_titles = df['title'].str.split('_').str[0].unique()

print(unique_titles)

Extract media type and remove it from title column

In [ ]:
df['media_type'] = df['title'].str.split('_').str[0]  # Extract media type
df['title'] = df['title'].str.split('_').str[1]       # Remove media type from title

df.head()

In [ ]:
print(df['media_type'].unique())

#### Create new df with media_type 'Movie' only

In [ ]:
doesthe_movies = df[df['media_type'] == 'Movie']
print(doesthe_movies)

In [ ]:
doesthe_movies.to_csv('../data/local/raw/doesthedog_movies.csv', index=False)

In [ ]:
print(f'Unique titles in doesthe_movies: {doesthe_movies['title'].nunique()}')

In [ ]:
doesthe_movies['clean_title'] = functions.prepare_clean_titles(doesthe_movies, 'title')
doesthe_movies.head()


In [ ]:
functions.show_basic_info(doesthe_movies)

Drop media_type column

In [ ]:
doesthe_movies.drop(columns=['media_type'], inplace=True)

#### Extract category_name unique values

In [ ]:
unique_values = sorted(doesthe_movies['category_name'].dropna().unique())
print(f'Unique values in category column: {unique_values}')

In [ ]:
print(f'Unique titles in doesthe_movies: {doesthe_movies['title'].nunique()}')

### Drop rows where 'yes' is greater than 'no'
Keep rows where sensitive content does take place.

In [ ]:
events_df = doesthe_movies.copy()
events_df = events_df[events_df['yes'] >= events_df['no']]

Group categories in new column

In [ ]:
events_df['events'] = events_df.groupby('clean_title')['category_name'].transform(lambda x: ', '.join(x))
events_df = events_df.drop(columns=['category_name'])

In [ ]:
events_df.head(10)

In [ ]:
events_df

### Check and remove duplicates

In [ ]:
functions.check_for_duplicates(events_df)

In [ ]:
functions.remove_duplicates(events_df)

In [ ]:
# Save the DataFrame to a CSV file, reset the index, and exclude the index column
events_df.reset_index(drop=True, inplace=True)
events_df.to_csv('../data/local/clean/doesthe_movie_events_df.csv', index=False)